In [1]:
import rascaline
rascaline._c_lib._get_library()

from copy import deepcopy

import numpy as np

from equisolve.numpy.scripts import MultiSpectraScript
from equisolve.numpy.models.linear_model import Ridge
from equisolve.numpy.preprocessing import StandardScaler
from equisolve.utils.convert import ase_to_tensormap

import ase.io


In [2]:
frames = ase.io.read("water_converted.extxyz", ":20")

In [3]:
HYPERS = {
    "cutoff": 3.0,
    "max_radial": 6,
    "max_angular": 4,
    "atomic_gaussian_width": 0.3,
    "center_atom_weight": 1.0,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
}
multi_spectra_hypers = {}
#multi_spectra_hypers['Composition'] = {}
multi_spectra_hypers['SoapRadialSpectrum'] = deepcopy(HYPERS)
multi_spectra_hypers['SoapRadialSpectrum'].pop('max_angular')
multi_spectra_hypers['SoapPowerSpectrum'] = deepcopy(HYPERS)

In [4]:
y = ase_to_tensormap(frames, energy="TotEnergy", forces="force")

In [5]:
# Because all frames have the same species, we cannot scale, so we just subtract the mean
#transformer_composition = StandardScaler(parameter_keys=["values", "positions"], with_mean=True, with_std=False)
# THIS IS BUGGY, something wrong in the standardizer when setting these flags to False

transformer = StandardScaler(parameter_keys=["values", "positions"])
estimator = Ridge(parameter_keys=["values", "positions"])
script = MultiSpectraScript(multi_spectra_hypers,
                            #transformer_X=transformer,
                            #transformer_y=transformer,
                            estimator=estimator)
Xi = script.compute(systems=frames, gradients=["positions"])
script.fit(Xi, y, **{"estimator_kwargs": {"alpha": 1e-15}})
script.score(Xi, y)

671.9636307759672

In [6]:
import pickle
with open("multi_spectra_script-water.pickle", "wb") as file:
    pickle.dump(script, file)

In [1]:
# RESTART NOTEBOOK

In [2]:
import pickle
with open("multi_spectra_script-water.pickle", "rb") as file:
    script = pickle.load(file)

In [5]:
# in ipi

import ase.io
import numpy as np

structure = ase.io.read("h5o2+.extxyz", "0")

Xi = script.compute(systems=structure, gradients=["positions"])
y_pred = script.forward(Xi) # implicitely done in score function
energy = y_pred.block().values[0][0]
forces = np.array(y_pred.block().gradient("positions").data.reshape(-1, 3))

print(energy)
print()
print(forces)

-29201.186821558364

[[-295.60671847   -4.62718863  -55.59864334]
 [ 298.54620907    4.04942317   37.15507667]
 [-221.68776379 -150.28232975 -242.81915489]
 [  -7.67013877    1.6028996    48.61326349]
 [ 176.03569875 -146.94124088  179.13481445]
 [ 286.69810363  136.65180697 -172.50009028]
 [-220.83316507  156.29491821  107.1942928 ]]


In [13]:
y_pred.block().properties.names

('property',)

In [10]:
from equisolve.numpy.scripts import GenericMDCalculator

from ipi.utils.mathtools import det_ut3x3
from ipi.utils.units import unit_to_internal, unit_to_user


md_calc = GenericMDCalculator("multi_spectra_script-water.pickle",
                    is_periodic=True,
                    structure_template="h5o2+.extxyz",
                    atomic_numbers=[1,8])

In [ ]:
from ipi.utils.mathtools import det_ut3x3
from ipi.utils.units import unit_to_internal, unit_to_user

pos_rascal = unit_to_user("length", "angstrom", pos)
# librascal expects ASE-format, cell-vectors-as-rows
cell_rascal = unit_to_user("length", "angstrom", cell.T)

pot, force, stress = md_calc.calculate(pos_rascal, cell_rascal)
